In [1]:
from owslib.wms import WebMapService

In [2]:
wms = WebMapService('https://image.discomap.eea.europa.eu/arcgis/services/GioLand/VHR_2021_LAEA/ImageServer/WMSServer/?request=GetCapabilities&service=WMS', version='1.3.0')

In [7]:
print(f"{wms.identification.title} - {wms.identification.version}")

VHR2021 seamless mosaic - 1.3.0


In [20]:
list(wms.contents)

['VHR_2021_LAEA']

In [21]:
layer_name = list(wms.contents)[0]

In [34]:
wms[layer_name].boundingBox

(-55.043848, 24.740774, 71.22644, 71.944746, 'CRS:84')

In [13]:
wms["VHR_2021_LAEA"].crsOptions   

['EPSG:3035', 'CRS:84', 'EPSG:4326']

In [16]:
[op.name for op in wms.operations]

['GetCapabilities', 'GetMap']

In [44]:
image_width = 1000
image_height = image_width / 1.7779953980476724

In [46]:
img = wms.getmap(
    layers=[layer_name],
    srs="CRS:84",
    bbox=wms[layer_name].boundingBoxWGS84,
    size=(image_width, image_height),
    format="image/jpeg",
    transparent=True,
)
with open("test_image_europa.jpg", "wb") as out:
    out.write(img.read())

# Calculating Correct Aspect Ratio for Geographic Bounding Box

To display geographic data without distortion, we need to account for the fact that longitude degrees get smaller as we move away from the equator.

In [39]:
import math

# Your bounding box (minx, miny, maxx, maxy) in WGS84
bbox = (-55.043848, 24.740774, 71.22644, 71.944746)
min_lon, min_lat, max_lon, max_lat = bbox

print(f"Bounding box:")
print(f"Longitude: {min_lon:.6f}° to {max_lon:.6f}° (span: {max_lon - min_lon:.6f}°)")
print(f"Latitude:  {min_lat:.6f}° to {max_lat:.6f}° (span: {max_lat - min_lat:.6f}°)")

Bounding box:
Longitude: -55.043848° to 71.226440° (span: 126.270288°)
Latitude:  24.740774° to 71.944746° (span: 47.203972°)


In [40]:
# Calculate the center latitude for longitude correction
center_lat = (min_lat + max_lat) / 2
center_lat_rad = math.radians(center_lat)

print(f"Center latitude: {center_lat:.6f}° ({center_lat:.1f}°)")

# At this latitude, how much does 1 degree of longitude shrink?
lon_correction_factor = math.cos(center_lat_rad)
print(f"Longitude correction factor: {lon_correction_factor:.6f}")

# Calculate real-world distances
lat_span_degrees = max_lat - min_lat
lon_span_degrees = max_lon - min_lon

# Convert to approximate meters (using 111,320 m per degree)
meters_per_degree = 111320

lat_span_meters = lat_span_degrees * meters_per_degree
lon_span_meters = lon_span_degrees * meters_per_degree * lon_correction_factor

print(f"\nReal-world dimensions:")
print(f"North-South (latitude):  {lat_span_meters/1000:.1f} km")
print(f"East-West (longitude):   {lon_span_meters/1000:.1f} km")

Center latitude: 48.342760° (48.3°)
Longitude correction factor: 0.664673

Real-world dimensions:
North-South (latitude):  5254.7 km
East-West (longitude):   9342.9 km


In [41]:
# Calculate the correct aspect ratio
# Aspect ratio = width / height
aspect_ratio = lon_span_meters / lat_span_meters

print(f"\nCorrect aspect ratio:")
print(f"Width/Height = {aspect_ratio:.6f}")
print(f"This means: {aspect_ratio:.2f}:1 (width:height)")

# If we want the height to be 1000 pixels, what should the width be?
target_height = 1000
correct_width = int(target_height * aspect_ratio)

print(f"\nFor undistorted imagery:")
print(f"If height = {target_height} pixels")
print(f"Then width should = {correct_width} pixels")
print(f"Image size: {correct_width} × {target_height}")

# Alternative: if we want width to be 1000, what should height be?
target_width = 1000
correct_height = int(target_width / aspect_ratio)
print(f"\nAlternatively:")
print(f"If width = {target_width} pixels")
print(f"Then height should = {correct_height} pixels")
print(f"Image size: {target_width} × {correct_height}")


Correct aspect ratio:
Width/Height = 1.777995
This means: 1.78:1 (width:height)

For undistorted imagery:
If height = 1000 pixels
Then width should = 1777 pixels
Image size: 1777 × 1000

Alternatively:
If width = 1000 pixels
Then height should = 562 pixels
Image size: 1000 × 562


## Comparison: Square vs Correct Aspect Ratio

In [ ]:
print("Comparison of image dimensions:")
print(f"Your current square image: 1000 × 1000")
print(f"  - Each pixel represents: {lat_span_meters/1000:.1f}m N-S, {lon_span_meters/1000:.1f}m E-W")
print(f"  - Distortion factor: {lon_span_meters/lat_span_meters:.2f}× wider than reality")
print()
print(f"Correct aspect ratio image: {correct_width} × {target_height}")  
print(f"  - Each pixel represents: {lat_span_meters/target_height:.1f}m N-S, {lon_span_meters/correct_width:.1f}m E-W")
print(f"  - No distortion: true geographic proportions")
print()
print(f"Alternative correct image: {target_width} × {correct_height}")
print(f"  - Each pixel represents: {lat_span_meters/correct_height:.1f}m N-S, {lon_span_meters/target_width:.1f}m E-W")
print(f"  - No distortion: true geographic proportions")

## Updated WMS Request with Correct Aspect Ratio

In [ ]:
# Example of how to request the image with correct proportions
correct_bbox = bbox  # Same bounding box: (-55.043848, 24.740774, 71.22644, 71.944746)

print("Updated WMS request with correct aspect ratio:")
print(f"bbox={correct_bbox}")
print(f"size=({correct_width}, {target_height})  # Instead of (1000, 1000)")
print()
print("This will give you a geographically accurate image where:")
print("- Distances are proportional in all directions") 
print("- Circles appear as circles (not ellipses)")
print("- Geographic features have their true shape")

# You can use either dimension option:
print(f"\nOption 1: {correct_width} × {target_height} pixels")
print(f"Option 2: {target_width} × {correct_height} pixels")
print("Both maintain the correct aspect ratio!")

## Calculate Image Size for Specific Pixel Resolution (2m × 2m per pixel)

In [47]:
# Target pixel resolution: 2m × 2m per pixel
target_pixel_size_meters = 2

print(f"Target: Each pixel represents {target_pixel_size_meters} × {target_pixel_size_meters} meters")
print()

# Calculate required image dimensions
# Width in pixels = total width in meters / meters per pixel
required_width_pixels = int(lon_span_meters / target_pixel_size_meters)
required_height_pixels = int(lat_span_meters / target_pixel_size_meters)

print(f"Required image dimensions for 2m×2m pixel resolution:")
print(f"Width:  {required_width_pixels:,} pixels")
print(f"Height: {required_height_pixels:,} pixels")
print(f"Total:  {required_width_pixels:,} × {required_height_pixels:,} = {required_width_pixels * required_height_pixels:,} pixels")

# Convert to megapixels
megapixels = (required_width_pixels * required_height_pixels) / 1_000_000
print(f"Image size: {megapixels:.1f} megapixels")

# File size estimation (rough)
# Assuming JPEG compression ~3 bytes per pixel on average
estimated_file_size_mb = (required_width_pixels * required_height_pixels * 3) / (1024 * 1024)
print(f"Estimated file size: ~{estimated_file_size_mb:.1f} MB (JPEG)")

Target: Each pixel represents 2 × 2 meters

Required image dimensions for 2m×2m pixel resolution:
Width:  4,671,457 pixels
Height: 2,627,373 pixels
Total:  4,671,457 × 2,627,373 = 12,273,659,992,461 pixels
Image size: 12273660.0 megapixels
Estimated file size: ~35115222.9 MB (JPEG)


In [ ]:
# Verify the aspect ratio is maintained
calculated_aspect_ratio = required_width_pixels / required_height_pixels
print(f"\nAspect ratio check:")
print(f"Original geographic aspect ratio: {aspect_ratio:.6f}")
print(f"Calculated image aspect ratio:   {calculated_aspect_ratio:.6f}")
print(f"Difference: {abs(aspect_ratio - calculated_aspect_ratio):.6f} (should be very small)")

print(f"\nActual pixel resolution:")
actual_pixel_width = lon_span_meters / required_width_pixels
actual_pixel_height = lat_span_meters / required_height_pixels
print(f"Width resolution:  {actual_pixel_width:.2f} m/pixel")
print(f"Height resolution: {actual_pixel_height:.2f} m/pixel")
print(f"Target was: {target_pixel_size_meters:.1f} m/pixel")

### Comparison with Different Pixel Sizes

In [48]:
# Compare different pixel resolutions
pixel_sizes = [1, 2, 5, 10, 20, 50, 100]  # meters per pixel

print("Comparison of different pixel resolutions:")
print("Pixel Size | Image Width × Height | Megapixels | Est. File Size")
print("-" * 65)

for pixel_size in pixel_sizes:
    width = int(lon_span_meters / pixel_size)
    height = int(lat_span_meters / pixel_size)
    megapx = (width * height) / 1_000_000
    file_size_mb = (width * height * 3) / (1024 * 1024)
    
    print(f"{pixel_size:>2}m × {pixel_size}m | {width:>5,} × {height:>5,} | {megapx:>8.1f} | {file_size_mb:>8.1f} MB")

print("\nNote: File sizes are rough estimates for JPEG format")
print("Actual file sizes depend on image complexity and compression settings")

Comparison of different pixel resolutions:
Pixel Size | Image Width × Height | Megapixels | Est. File Size
-----------------------------------------------------------------
 1m × 1m | 9,342,914 × 5,254,746 | 49094640.0 | 140460891.6 MB
 2m × 2m | 4,671,457 × 2,627,373 | 12273660.0 | 35115222.9 MB
 5m × 5m | 1,868,582 × 1,050,949 | 1963784.4 | 5618432.2 MB
10m × 10m | 934,291 × 525,474 | 490945.6 | 1404606.7 MB
20m × 20m | 467,145 × 262,737 | 122736.3 | 351151.3 MB
50m × 50m | 186,858 × 105,094 |  19637.7 |  56183.8 MB
100m × 100m | 93,429 × 52,547 |   4909.4 |  14045.9 MB

Note: File sizes are rough estimates for JPEG format
Actual file sizes depend on image complexity and compression settings


### WMS Request for 2m Resolution

In [ ]:
print("To get 2m × 2m pixel resolution, use:")
print(f"size=({required_width_pixels}, {required_height_pixels})")
print()
print("Full WMS request example:")
print(f"""
img = wms.getmap(
    layers=[layer_name],
    srs="CRS:84", 
    bbox=({bbox[0]}, {bbox[1]}, {bbox[2]}, {bbox[3]}),
    size=({required_width_pixels}, {required_height_pixels}),
    format="image/jpeg",
    transparent=True,
)
""")

print("⚠️  WARNING:")
print(f"This will create a {megapixels:.1f} megapixel image (~{estimated_file_size_mb:.1f} MB)")
print("Make sure your system has enough memory and the WMS server allows such large requests!")
print("\nConsider using a smaller area or lower resolution for testing first.")